In [2]:
from collections import OrderedDict

import yaml
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
# from torchvision.datasets import CIFAR10
import flwr as fl
from datetime import datetime
import importlib
import os
import sys
sys.path.append('/home/akis-linardos/BFP/src')
from models import nets
from data_loader import ALLDataset
from tqdm import tqdm
import numpy as np
from pathlib import Path
import pickle
# from sklearn import roc_auc_score
# import argparse

In [11]:
def import_class(name):
    module_name, class_name = name.rsplit('.', 1)
    module = importlib.import_module(module_name)
    return getattr(module, class_name)

HOME_PATH = Path.home()

config_file = 'config.yaml'
with open(config_file) as file:
  CONFIG = yaml.safe_load(file)

CSV_PATH = os.environ['csv_path']
DATASET_PATH = os.environ['dataset_path']
# Before running each client locally, you need to set the environment variable client_log_path to a unique value for each worker

DATA_LOADER_TYPE= os.getenv('data_loader_type',"optimam") #env variable data_loader if not given default to optimam type dataloading

# Docker ip is: 172.17.0.3
print(f'Here dataset path {DATASET_PATH}')
print(f'Here csv path {CSV_PATH}')

DEVICE = torch.device(CONFIG['device']) #if torch.cuda.is_available() else "cpu")
CRITERION = import_class(CONFIG['hyperparameters']['criterion'])


log_dict = {'local_loss':{0:[]}, 'local_val_loss':{0:[]}, 'local_accuracy':[], 'local_sensitivity':[], 'local_specificity':[], 'local_val_predictions':[],
            'local_true_positives':[],'local_false_positives':[],'local_false_negatives':[],'local_true_negatives':[]}

Here dataset path /home/akis-linardos/Datasets
Here csv path /home/akis-linardos/Datasets/CMMD/info.csv


In [5]:
def load_data():
    """Load Breast Cancer training and validation set."""
    print('Loading data...')
    training_loader = DataLoader(ALLDataset(DATASET_PATH, CSV_PATH, mode='train', data_loader_type=DATA_LOADER_TYPE, load_max=CONFIG['data']['load_max']), batch_size=CONFIG['hyperparameters']['batch_size'])
    validation_loader = DataLoader(ALLDataset(DATASET_PATH, CSV_PATH, mode='val', data_loader_type=DATA_LOADER_TYPE, load_max=CONFIG['data']['load_max']), batch_size=CONFIG['hyperparameters']['batch_size'])
    test_loader = DataLoader(ALLDataset(DATASET_PATH, CSV_PATH, mode='test', data_loader_type=DATA_LOADER_TYPE, load_max=CONFIG['data']['load_max']), batch_size=CONFIG['hyperparameters']['batch_size'])
    print(len(training_loader))
    return training_loader, validation_loader #test_loader

def train(net, training_loader, criterion):
    """Train the network on the training set."""
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    losses = []
    cumulative_loss = 0.0
    predictions = []
    print('Training...')
    current_round_num=list(log_dict['local_loss'].keys())[-1]
    next_round_num=current_round_num+1
    log_dict['local_loss'][next_round_num]=[] # A key for the next round is generated. Final round will always remain empty
    for _ in range(CONFIG['hyperparameters']['epochs_per_round']):
        for i, batch in enumerate(tqdm(training_loader)):
            images, labels = batch[0].to(DEVICE), batch[1].to(DEVICE).unsqueeze(1)
            optimizer.zero_grad()
            loss = criterion(net(images), labels)
            cumulative_loss += loss.item()
            loss.backward()
            optimizer.step()
    train_results = cumulative_loss #(losses, predictions)
    return train_results

In [18]:
def probabilities_to_labels(predictions : torch.Tensor) -> torch.Tensor:
    """Convert the network's predictions to labels."""
    if predictions.size()[1]==1: # if not one hot encoding
        return torch.round(predictions) #sigmoid outputs
    predictions = predictions.detach().numpy()
    predictions_as_labels = []
    for row in predictions:
        predictions_as_labels.append(np.argmax(row))
    return torch.Tensor(predictions_as_labels)

def test(net, validation_loader, criterion):
    """Validate the network on the entire test set."""
    correct, total, cumulative_loss = 0, 0, 0.0
    false_positive, false_negative, true_positive, true_negative = 0, 0, 0, 0
    predictions, val_losses = [], []
    current_round_num=list(log_dict['local_val_loss'].keys())[-1]
    next_round_num=current_round_num+1
    log_dict['local_val_loss'][next_round_num]=[] # A key for the next round is generated. Final round will always remain empty
    with torch.no_grad():
        for i, batch in enumerate(tqdm(validation_loader)):
            images, labels = batch[0].to(DEVICE), batch[1].to(DEVICE).unsqueeze(1)
            outputs = net(images)
            loss = criterion(outputs, labels).item()
            cumulative_loss += criterion(outputs, labels).item()
            predicted = probabilities_to_labels(outputs.data)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            false_positive += ((predicted == 1) & (labels == 0)).sum().item()
            false_negative += ((predicted == 0) & (labels == 1)).sum().item()
            true_positive += ((predicted == 1) & (labels == 1)).sum().item()
            true_negative += ((predicted == 0) & (labels == 0)).sum().item()
            predictions.append(predicted)
            val_losses.append(loss)
    val_loss=sum(val_losses)/len(val_losses)
    log_dict['local_val_loss'][current_round_num]=val_loss
    accuracy = correct / total
    # sensitivity = true_positive.sum().item() / (true_positive.sum().item() + false_negative.sum().item())
    # specificity = true_negative.sum().item() / (true_negative.sum().item() + false_positive.sum().item())
    # AUC = roc_auc_score(labels.detach().numpy(), outputs.detach().numpy())
    log_dict['local_accuracy'].append(accuracy)
    # log_dict['local_sensitivity'].append(sensitivity)
    # log_dict['local_specificity'].append(specificity)
    # Store everything!
    log_dict['local_true_positives'].append(true_positive)
    log_dict['local_true_negatives'].append(true_negative)
    log_dict['local_false_positives'].append(false_positive)
    log_dict['local_false_negatives'].append(false_negative)

    loss = cumulative_loss / total
    # import pdb; pdb.set_trace()
    # test_results = (loss, accuracy, bytes(predictions))
    test_results = (loss, accuracy)
    return test_results

In [19]:
train_loader, validation_loader = load_data()


Loading data...
Total images selected by status (benign): 1108
Total images selected by status (malignant): 1108
Total images selected by status (benign): 1108
Total images selected by status (malignant): 1108
Total images selected by status (benign): 1108
Total images selected by status (malignant): 1108
178


In [8]:
net = nets.ResNet101Classifier(in_ch=3, out_ch=1, pretrained=False)
net.to(DEVICE)
criterion=CRITERION()

Using cache found in /home/akis-linardos/.cache/torch/hub/pytorch_vision_v0.10.0


In [20]:
test_results = test(net, validation_loader, criterion)

100%|██████████████████████████████████████████| 45/45 [00:07<00:00,  6.13it/s]


In [21]:
test_results

(0.09248985907247474, 0.509009009009009)